In [2]:
import pandas as pd
import xgboost as xg
from sklearn.metrics import classification_report, confusion_matrix, ConfusionMatrixDisplay
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier

In [10]:
# Attempt with 'latin1' encoding
try:
    loandata = pd.read_csv('Loan Status prediction data.xlsx - Query result.csv', encoding='latin1')
    print("File read successfully with 'latin1' encoding.")
except UnicodeDecodeError:
    print("UnicodeDecodeError with 'latin1' encoding. Trying next encoding...")

# Attempt with 'iso-8859-1' encoding
try:
    loandata = pd.read_csv('Loan Status prediction data.xlsx - Query result.csv', encoding='iso-8859-1')
    print("File read successfully with 'iso-8859-1' encoding.")
except UnicodeDecodeError:
    print("UnicodeDecodeError with 'iso-8859-1' encoding. Trying next encoding...")

# Attempt with 'cp1252' encoding
try:
    loandata = pd.read_csv('Loan Status prediction data.xlsx - Query result.csv', encoding='cp1252')
    print("File read successfully with 'cp1252' encoding.")
except UnicodeDecodeError:
    print("UnicodeDecodeError with 'cp1252' encoding. Please check the file encoding.")

File read successfully with 'latin1' encoding.
File read successfully with 'iso-8859-1' encoding.
File read successfully with 'cp1252' encoding.


In [11]:
loandata.head()

,org_id,user_id,loan_id,gender,marital_status,no_of_dependent,type_of_residence,educational_attainment,employment_status,sector_of_employment,...,emails,phone_numbers,logins,device_name,mobile_os,os_version,bank,created_on,process_time,status_id
0,2,"1,803,407","203,740",Male,Single,0,Own House,"BSc, HND and Other Equivalent",Self Employed,Oil and Gas,...,1,1,6,TECNO MOBILE LIMITED TECNO LC8,android,android 29,33.0,"Apr 28, 2024, 12:44 PM","Apr 29, 2024, 7:29 PM",5
1,2,"1,802,930","203,723",Male,Single,0,Rented Apartment,School Cert,Employed,Others,...,1,1,6,samsung SM-J610F,android,android 28,57.0,"Apr 28, 2024, 10:44 AM","Apr 29, 2024, 7:29 PM",5
2,2,"1,802,647","203,685",Female,Married,2,Rented Apartment,Diploma,Self Employed,Wholesale and Retail Trade,...,1,2,11,TECNO MOBILE LIMITED TECNO BA2,android,android 27,11.0,"Apr 27, 2024, 11:18 PM","Apr 29, 2024, 7:29 PM",2
3,2,"1,802,295","203,614",Male,Single,0,Rented Apartment,MSc and Above,Employed,Manufacturing and Construction,...,1,1,3,OPPO PEMM00,android,android 33,58.0,"Apr 27, 2024, 1:11 PM","Apr 29, 2024, 7:29 PM",5
4,2,"1,802,173","203,591",Male,Single,0,Own House,"BSc, HND and Other Equivalent",Self Employed,Agriculture,...,1,1,2,NaN,NaN,NaN,33.0,"Apr 27, 2024, 11:44 AM","Apr 29, 2024, 7:29 PM",2


In [12]:
print(f"Shape of the dataset: {loandata.shape}")

Shape of the dataset: (11847, 45)


In [13]:
print(f"Informations About The Dataset :\n")
print(loandata.info())

#this contains information such as variable types, memory usage, etc.

Informations About The Dataset :

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11847 entries, 0 to 11846
Data columns (total 45 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   org_id                    11847 non-null  int64  
 1   user_id                   11847 non-null  object 
 2   loan_id                   11847 non-null  object 
 3   gender                    10593 non-null  object 
 4   marital_status            11847 non-null  object 
 5   no_of_dependent           11847 non-null  object 
 6   type_of_residence         11828 non-null  object 
 7   educational_attainment    11828 non-null  object 
 8   employment_status         11828 non-null  object 
 9   sector_of_employment      11828 non-null  object 
 10  current_employer          11736 non-null  object 
 11  monthly_net_income        11828 non-null  object 
 12  work_start_date           11846 non-null  object 
 13  work_email                1

In [14]:
loandata.describe(include=object).T.style.set_properties(**{"color":"blue","border": "1.2px  solid black"})

,count,unique,top,freq
user_id,11847,4337,"319,150",23
loan_id,11847,11847,"127,846",1
gender,10593,2,Male,7905
marital_status,11847,5,Single,7322
no_of_dependent,11847,4,0,7157
type_of_residence,11828,3,Rented Apartment,7045
educational_attainment,11828,8,"BSc, HND and Other Equivalent",7493
employment_status,11828,4,Self Employed,6954
sector_of_employment,11828,18,Others,2383
current_employer,11736,3502,Self,321


In [15]:
print("Null Values of Dataset :")
loandata.isna().sum().to_frame().T.style.set_properties(**{"color":"blue","border": "1.5px  solid black"})

Null Values of Dataset :


,org_id,user_id,loan_id,gender,marital_status,no_of_dependent,type_of_residence,educational_attainment,employment_status,sector_of_employment,current_employer,monthly_net_income,work_start_date,work_email,work_email_validated,country,city,lga,requested_amount,purpose,proposed_payday,loan_request_hour,loan_request_day,age,selfie_bvn_check,selfie_id_check,phone_network,loan_requests,failed_loan_requests,successful_loan_requests,loans,lending_lenders,lenders,first_account,last_account,emails,phone_numbers,logins,device_name,mobile_os,os_version,bank,created_on,process_time,status_id
0,0,0,0,1254,0,0,19,19,19,19,111,19,1,29,0,0,363,877,0,0,14,0,0,0,35,53,0,0,0,0,0,0,0,0,0,0,0,0,377,363,377,1,0,0,0


In [16]:
loandata.dropna(axis = 0, inplace = True)
loandata.isna().sum().to_frame().T.style.set_properties(**{"color":"blue","border": "1.5px  solid black"})

,org_id,user_id,loan_id,gender,marital_status,no_of_dependent,type_of_residence,educational_attainment,employment_status,sector_of_employment,current_employer,monthly_net_income,work_start_date,work_email,work_email_validated,country,city,lga,requested_amount,purpose,proposed_payday,loan_request_hour,loan_request_day,age,selfie_bvn_check,selfie_id_check,phone_network,loan_requests,failed_loan_requests,successful_loan_requests,loans,lending_lenders,lenders,first_account,last_account,emails,phone_numbers,logins,device_name,mobile_os,os_version,bank,created_on,process_time,status_id
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [17]:
print(f"Shape of the dataset: {loandata.shape}")

Shape of the dataset: (9409, 45)


In [18]:
loandata.head()

,org_id,user_id,loan_id,gender,marital_status,no_of_dependent,type_of_residence,educational_attainment,employment_status,sector_of_employment,...,emails,phone_numbers,logins,device_name,mobile_os,os_version,bank,created_on,process_time,status_id
0,2,"1,803,407","203,740",Male,Single,0,Own House,"BSc, HND and Other Equivalent",Self Employed,Oil and Gas,...,1,1,6,TECNO MOBILE LIMITED TECNO LC8,android,android 29,33.0,"Apr 28, 2024, 12:44 PM","Apr 29, 2024, 7:29 PM",5
1,2,"1,802,930","203,723",Male,Single,0,Rented Apartment,School Cert,Employed,Others,...,1,1,6,samsung SM-J610F,android,android 28,57.0,"Apr 28, 2024, 10:44 AM","Apr 29, 2024, 7:29 PM",5
2,2,"1,802,647","203,685",Female,Married,2,Rented Apartment,Diploma,Self Employed,Wholesale and Retail Trade,...,1,2,11,TECNO MOBILE LIMITED TECNO BA2,android,android 27,11.0,"Apr 27, 2024, 11:18 PM","Apr 29, 2024, 7:29 PM",2
3,2,"1,802,295","203,614",Male,Single,0,Rented Apartment,MSc and Above,Employed,Manufacturing and Construction,...,1,1,3,OPPO PEMM00,android,android 33,58.0,"Apr 27, 2024, 1:11 PM","Apr 29, 2024, 7:29 PM",5
5,2,"1,712,673","203,536",Female,Married,2,Rented Apartment,Diploma,Self Employed,Oil and Gas,...,1,1,22,TECNO MOBILE LIMITED TECNO LB8,android,android 28,33.0,"Apr 27, 2024, 12:39 AM","Apr 29, 2024, 7:29 PM",2


In [19]:

target_encoding = loandata['status_id'].unique()



print(target_encoding)

[5 2]


In [20]:
def convert_to_int(value):
    if value == '3 or more':
        return 3  # or any other suitable value that represents '3 or more'
    else:
        try:
            return int(value)
        except ValueError:
            return 0  # or any other default value if conversion fails

def classify_loan_score(score):
    if score >= 65:
        return 'True'
    elif score >= 50:
        return 'Maybe'
    else:
        return 'False'

def calculate_combined_loan_score(row):
    # Extracting variables for weighted loan score calculation
    marital_status = row['marital_status']
    no_of_dependent = convert_to_int(row['no_of_dependent'])  # Convert to integer
    type_of_residence = row['type_of_residence']
    educational_attainment = row['educational_attainment']
    employment_status = row['employment_status']
    sector_of_employment = row['sector_of_employment']

    # Handling special case for 'Parents Apartment'
    if type_of_residence == 'Parents Apartment':
        type_of_residence = 'Rented Apartment'

    # Defining weights for weighted loan score calculation
    marital_weights = {'Single': 0.5, 'Married': 0.7, 'Others': 0.3}
    dependent_weights = {0: 1, 1: 0.9, 2: 0.8, 3: 0.7, 4: 0.6, 5: 0.5, 6: 0.4, 7: 0.3, 8: 0.2, 9: 0.1}
    residence_weights = {'Own House': 1, 'Rented Apartment': 0.8}
    education_weights = {'MSc and Above': 1, 'BSc, HND and Other Equivalent': 0.9, 'Diploma': 0.8, 'School Cert': 0.7, 'Vocation/Technical': 0.7}  # Add a default weight for the missing value
    employment_weights = {'Employed': 1, 'Self Employed': 0.9, 'Unemployed': 0.3}
    sector_weights = {'Oil and Gas': 1, 'Banking and Finance': 0.9, 'Manufacturing and Construction': 0.8, 'Wholesale and Retail Trade': 0.7}

    # Handling the case when value is not found in the dictionary
    marital_weight = marital_weights.get(marital_status, 0.5)  # Default to 0.5 if not found
    sector_weight = sector_weights.get(sector_of_employment, 0.7)  # Default to 0.7 if not found

    # Calculating weighted loan score
    weighted_sum = (marital_weight +
                    dependent_weights[no_of_dependent] +
                    residence_weights[type_of_residence] +
                    education_weights.get(educational_attainment, 0.7) +  # Default to 0.7 if not found
                    employment_weights.get(employment_status, 0.5) +  # Default to 0.5 if not found
                    sector_weight)

    weighted_loan_score = (weighted_sum / 6) * 100

    # Calculating original loan score
    total_requests = row['loan_requests']
    failed_requests = row['failed_loan_requests']
    successful_requests = row['successful_loan_requests']

    if total_requests == 0:
        original_loan_score = 0
    else:
        success_rate = successful_requests / total_requests
        if failed_requests == 0:
            failure_adjustment = 1
        else:
            failure_adjustment = (1 - (failed_requests / total_requests))

        original_loan_score = (success_rate * failure_adjustment) * 100

    # Assigning weight to the original loan score
    loan_score_weight = 0.5  # You can adjust this weight as needed

    # Combining weighted loan score and original loan score with weights
    combined_score = (loan_score_weight * original_loan_score) + ((1 - loan_score_weight) * weighted_loan_score)

    # Classifying the combined score into a loan grade
    loan_grade = classify_loan_score(combined_score)

    return loan_grade

# Create a new column 'loan_grade' based on the classification of combined loan scores
loandata['loan_grade'] = loandata.apply(calculate_combined_loan_score, axis=1)






In [21]:
loandata.head()

,org_id,user_id,loan_id,gender,marital_status,no_of_dependent,type_of_residence,educational_attainment,employment_status,sector_of_employment,...,phone_numbers,logins,device_name,mobile_os,os_version,bank,created_on,process_time,status_id,loan_grade
0,2,"1,803,407","203,740",Male,Single,0,Own House,"BSc, HND and Other Equivalent",Self Employed,Oil and Gas,...,1,6,TECNO MOBILE LIMITED TECNO LC8,android,android 29,33.0,"Apr 28, 2024, 12:44 PM","Apr 29, 2024, 7:29 PM",5,False
1,2,"1,802,930","203,723",Male,Single,0,Rented Apartment,School Cert,Employed,Others,...,1,6,samsung SM-J610F,android,android 28,57.0,"Apr 28, 2024, 10:44 AM","Apr 29, 2024, 7:29 PM",5,False
2,2,"1,802,647","203,685",Female,Married,2,Rented Apartment,Diploma,Self Employed,Wholesale and Retail Trade,...,2,11,TECNO MOBILE LIMITED TECNO BA2,android,android 27,11.0,"Apr 27, 2024, 11:18 PM","Apr 29, 2024, 7:29 PM",2,False
3,2,"1,802,295","203,614",Male,Single,0,Rented Apartment,MSc and Above,Employed,Manufacturing and Construction,...,1,3,OPPO PEMM00,android,android 33,58.0,"Apr 27, 2024, 1:11 PM","Apr 29, 2024, 7:29 PM",5,False
5,2,"1,712,673","203,536",Female,Married,2,Rented Apartment,Diploma,Self Employed,Oil and Gas,...,1,22,TECNO MOBILE LIMITED TECNO LB8,android,android 28,33.0,"Apr 27, 2024, 12:39 AM","Apr 29, 2024, 7:29 PM",2,False


In [22]:
# Unique values of loan score
loan_score_values = loandata['loan_grade'].unique()


print("Unique values of loan score:", loan_score_values)


Unique values of loan score: ['False' 'Maybe' 'True']


In [23]:
loandata.tail()

,org_id,user_id,loan_id,gender,marital_status,no_of_dependent,type_of_residence,educational_attainment,employment_status,sector_of_employment,...,phone_numbers,logins,device_name,mobile_os,os_version,bank,created_on,process_time,status_id,loan_grade
11841,2,"668,118","127,870",Male,Single,0,Own House,"BSc, HND and Other Equivalent",Employed,Information Technology,...,1,41,TECNO MOBILE LIMITED TECNO KD7,android,android 29,58.0,"Jan 2, 2023, 1:28 PM","Apr 29, 2024, 7:29 PM",2,False
11842,2,"390,011","127,858",Male,Married,0,Rented Apartment,"BSc, HND and Other Equivalent",Self Employed,Agriculture,...,1,177,"iPhone11,2",ios,ios 16.0.2,57.0,"Jan 1, 2023, 5:59 PM","Apr 29, 2024, 7:29 PM",2,False
11843,2,"541,664","127,852",Female,Single,0,Parents Apartment,"BSc, HND and Other Equivalent",Self Employed,Wholesale and Retail Trade,...,1,85,"iPhone10,3",ios,ios 16.1,33.0,"Jan 1, 2023, 12:11 PM","Apr 29, 2024, 7:29 PM",2,Maybe
11844,2,"406,444","127,850",Male,Single,0,Rented Apartment,"BSc, HND and Other Equivalent",Self Employed,Transportation & Logistics,...,1,280,"iPhone10,3",ios,ios 15.6.1,57.0,"Jan 1, 2023, 9:04 AM","Apr 29, 2024, 7:29 PM",2,False
11845,2,"564,904","127,849",Male,Single,0,Parents Apartment,"BSc, HND and Other Equivalent",Employed,Public services and administration,...,1,215,"iPhone7,1",ios,ios 12.4.8,50211.0,"Jan 1, 2023, 9:03 AM","Apr 29, 2024, 7:29 PM",2,False


In [24]:
# Define the specific mapping for loan grades
loan_grade_mapping = {
    'True': 0,
    'Maybe': 1,
    'False': 2
}

# Apply the mapping to the 'loan_grade' column
loandata['loan_grade'] = loandata['loan_grade'].map(loan_grade_mapping)


In [25]:
loandata.head()

,org_id,user_id,loan_id,gender,marital_status,no_of_dependent,type_of_residence,educational_attainment,employment_status,sector_of_employment,...,phone_numbers,logins,device_name,mobile_os,os_version,bank,created_on,process_time,status_id,loan_grade
0,2,"1,803,407","203,740",Male,Single,0,Own House,"BSc, HND and Other Equivalent",Self Employed,Oil and Gas,...,1,6,TECNO MOBILE LIMITED TECNO LC8,android,android 29,33.0,"Apr 28, 2024, 12:44 PM","Apr 29, 2024, 7:29 PM",5,2
1,2,"1,802,930","203,723",Male,Single,0,Rented Apartment,School Cert,Employed,Others,...,1,6,samsung SM-J610F,android,android 28,57.0,"Apr 28, 2024, 10:44 AM","Apr 29, 2024, 7:29 PM",5,2
2,2,"1,802,647","203,685",Female,Married,2,Rented Apartment,Diploma,Self Employed,Wholesale and Retail Trade,...,2,11,TECNO MOBILE LIMITED TECNO BA2,android,android 27,11.0,"Apr 27, 2024, 11:18 PM","Apr 29, 2024, 7:29 PM",2,2
3,2,"1,802,295","203,614",Male,Single,0,Rented Apartment,MSc and Above,Employed,Manufacturing and Construction,...,1,3,OPPO PEMM00,android,android 33,58.0,"Apr 27, 2024, 1:11 PM","Apr 29, 2024, 7:29 PM",5,2
5,2,"1,712,673","203,536",Female,Married,2,Rented Apartment,Diploma,Self Employed,Oil and Gas,...,1,22,TECNO MOBILE LIMITED TECNO LB8,android,android 28,33.0,"Apr 27, 2024, 12:39 AM","Apr 29, 2024, 7:29 PM",2,2


In [26]:
loandata.tail()

,org_id,user_id,loan_id,gender,marital_status,no_of_dependent,type_of_residence,educational_attainment,employment_status,sector_of_employment,...,phone_numbers,logins,device_name,mobile_os,os_version,bank,created_on,process_time,status_id,loan_grade
11841,2,"668,118","127,870",Male,Single,0,Own House,"BSc, HND and Other Equivalent",Employed,Information Technology,...,1,41,TECNO MOBILE LIMITED TECNO KD7,android,android 29,58.0,"Jan 2, 2023, 1:28 PM","Apr 29, 2024, 7:29 PM",2,2
11842,2,"390,011","127,858",Male,Married,0,Rented Apartment,"BSc, HND and Other Equivalent",Self Employed,Agriculture,...,1,177,"iPhone11,2",ios,ios 16.0.2,57.0,"Jan 1, 2023, 5:59 PM","Apr 29, 2024, 7:29 PM",2,2
11843,2,"541,664","127,852",Female,Single,0,Parents Apartment,"BSc, HND and Other Equivalent",Self Employed,Wholesale and Retail Trade,...,1,85,"iPhone10,3",ios,ios 16.1,33.0,"Jan 1, 2023, 12:11 PM","Apr 29, 2024, 7:29 PM",2,1
11844,2,"406,444","127,850",Male,Single,0,Rented Apartment,"BSc, HND and Other Equivalent",Self Employed,Transportation & Logistics,...,1,280,"iPhone10,3",ios,ios 15.6.1,57.0,"Jan 1, 2023, 9:04 AM","Apr 29, 2024, 7:29 PM",2,2
11845,2,"564,904","127,849",Male,Single,0,Parents Apartment,"BSc, HND and Other Equivalent",Employed,Public services and administration,...,1,215,"iPhone7,1",ios,ios 12.4.8,50211.0,"Jan 1, 2023, 9:03 AM","Apr 29, 2024, 7:29 PM",2,2


In [27]:
loandata.info()


<class 'pandas.core.frame.DataFrame'>
Index: 9409 entries, 0 to 11845
Data columns (total 46 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   org_id                    9409 non-null   int64  
 1   user_id                   9409 non-null   object 
 2   loan_id                   9409 non-null   object 
 3   gender                    9409 non-null   object 
 4   marital_status            9409 non-null   object 
 5   no_of_dependent           9409 non-null   object 
 6   type_of_residence         9409 non-null   object 
 7   educational_attainment    9409 non-null   object 
 8   employment_status         9409 non-null   object 
 9   sector_of_employment      9409 non-null   object 
 10  current_employer          9409 non-null   object 
 11  monthly_net_income        9409 non-null   object 
 12  work_start_date           9409 non-null   object 
 13  work_email                9409 non-null   object 
 14  work_email_v

In [28]:


# Convert boolean variables to integers
bool_columns = loandata.select_dtypes(include='bool').columns
loandata[bool_columns] = loandata[bool_columns].astype(int)

# Handle Categorical Variables
# Use one-hot encoding for categorical variables
loandata = pd.get_dummies(loandata)



# Separate Features (X) and Target (Y)
X = loandata.drop(columns=['loan_grade'])  # Features
Y = loandata['loan_grade']  # Target


X.fillna(X.mean(), inplace=True)  # Filling missing values with the mean of each column




# Using Logistic Regression

In [29]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, stratify=Y, random_state=2)
print(X.shape, X_train.shape, X_test.shape)

(9409, 37757) (7527, 37757) (1882, 37757)


In [30]:
from sklearn.linear_model import LogisticRegression
model = LogisticRegression()

In [ ]:
model.fit(X_train, Y_train)

In [ ]:
from sklearn.metrics import accuracy_score

X_train_prediction = model.predict(X_train)
accuracy_train = accuracy_score(X_train_prediction, Y_train)

print('Accuracy on Training Data: ', accuracy_train )


X_test_prediction = model.predict(X_test)
accuracy_test = accuracy_score(X_test_prediction, Y_test)

print('Accuracy on Testing Data: ', accuracy_test )

# Using xgboost

In [ ]:
xg_model2 = xg.XGBClassifier(
    objective='multi:softprob',
    n_estimators=100,
    max_depth=10,
    learning_rate=0.01,
)

xg_model2.fit(X_train, Y_train)

In [ ]:
booster2 = xg_model2.get_booster()

feature_importance2 = booster2.get_score(importance_type='gain')

feature_importance2

In [ ]:
test_score = 100 * xg_model2.score(X_test, Y_test)

print(f'The model has an accuracy of {test_score:.2f}%')

In [ ]:
preds = xg_model2.predict(X_test)

print(classification_report(Y_test, preds))

In [ ]:
from sklearn.model_selection import cross_val_score

cv_score = cross_val_score(xg_model2, X_test, Y_test, cv=5)

cv_score

In [ ]:
# Save the trained model
xg_model2.save_model('loan_model.json')

In [ ]:
data = pd.read_csv('Model test data.csv')

data.head()


In [ ]:
# Create a copy of the original DataFrame before preprocessing
original_data = data.copy()

# Preprocess the data
# Drop rows with missing values
data.dropna(inplace=True)

# Encode categorical variables using one-hot encoding
data = pd.get_dummies(data)

# Ensure that the columns in the test data match the columns used during training
# If there are missing columns in the test data, you can add them and fill with zeros
missing_cols = set(X.columns) - set(data.columns)

missing_data = pd.DataFrame(0, index=data.index, columns=list(missing_cols))

# Concatenate the original data with the missing data
data = pd.concat([data, missing_data], axis=1)

# Get the columns used for training the logistic regression model
training_columns = X.columns

# Ensure that the columns in the input data match the training columns
data = data.reindex(columns=training_columns, fill_value=0)

# Make predictions using the logistic regression model
logistic_regression_predictions = model.predict(data)

# Make predictions using the XGBoost model
xgboost_predictions = xg_model2.predict(data)


# Print out the predicted loan scores
print("Predicted loan scores using Logistic Regression model:")
print(logistic_regression_predictions)

print("Predicted loan scores using XGBoost model:")
print(xgboost_predictions)



In [ ]:
original_data.head()

In [ ]:
# Assuming logistic_regression_predictions and xgboost_predictions hold the prediction results
logistic_regression_scores = logistic_regression_predictions
xgboost_scores = xgboost_predictions

# Define a dictionary mapping numeric values to their corresponding categories
score_mapping = {0: True, 1: 2.5, 2: False}

# Create a new DataFrame with user_id, loan_id, and index aligned with data
predicted_scores_data = original_data[['user_id', 'loan_id']].reindex(index=data.index)

# Add the predicted scores as new columns
predicted_scores_data['Logistic_Regression_Score'] = logistic_regression_scores
predicted_scores_data['XGBoost_Score'] = xgboost_scores

# Map the numeric scores to their corresponding categories
predicted_scores_data['Logistic_Regression_Score_Category'] = predicted_scores_data['Logistic_Regression_Score'].map(score_mapping)
predicted_scores_data['XGBoost_Score_Category'] = predicted_scores_data['XGBoost_Score'].map(score_mapping)

# Print out the predicted loan scores
print("Predicted loan scores using Logistic Regression model:")
print(predicted_scores_data[['user_id', 'loan_id', 'Logistic_Regression_Score', 'Logistic_Regression_Score_Category']])

print("Predicted loan scores using XGBoost model:")
print(predicted_scores_data[['user_id', 'loan_id', 'XGBoost_Score', 'XGBoost_Score_Category']])




In [ ]:
predicted_scores_data.head()

In [ ]:
predicted_scores_data.to_csv('predicted_scores.csv', index=False)

In [ ]:
from google.colab import files
uploaded = files.upload()

In [ ]:
newdata = pd.read_csv('Test data.xlsx - Query result.csv')

In [ ]:
newdata.head()

In [ ]:
# Copy the original data
originalnewdata = newdata.copy()

# Preprocess the data
# Drop rows with missing values
newdata.dropna(inplace=True)

# Encode categorical variables using one-hot encoding
newdata = pd.get_dummies(newdata)

# Ensure that the columns in the test data match the columns used during training
# If there are missing columns in the test data, you can add them and fill with zeros
missing_cols = set(X.columns) - set(newdata.columns)

missing_data = pd.DataFrame(0, index=newdata.index, columns=list(missing_cols))

# Concatenate the original data with the missing data
newdata = pd.concat([newdata, missing_data], axis=1)


# Remove extra columns that are not in the training data
extra_cols = set(newdata.columns) - set(X.columns)
newdata.drop(columns=extra_cols, inplace=True)

# Ensure the order of columns matches the training data
newdata = newdata[X.columns]

# Make predictions using the logistic regression model
new_logistic_regression_predictions = model.predict(newdata)

# Make predictions using the XGBoost model
new_xgboost_predictions = xg_model2.predict(newdata)

# Assuming logistic_regression_predictions and xgboost_predictions hold the prediction results
new_logistic_regression_scores = new_logistic_regression_predictions
new_xgboost_scores = new_xgboost_predictions

# Define a dictionary mapping numeric values to their corresponding categories
score_mapping = {0: True, 1: 2.5, 2: False}

# Create a new DataFrame with user_id, loan_id, and index aligned with data
predicted_scores_testdata = originalnewdata[['user_id', 'loan_id']].reindex(index=newdata.index)

# Add the predicted scores as new columns
predicted_scores_testdata['Logistic_Regression_Score'] = new_logistic_regression_scores
predicted_scores_testdata['XGBoost_Score'] = new_xgboost_scores

# Optionally map the numeric scores to their corresponding categories
predicted_scores_testdata['Logistic_Regression_Score_Category'] = predicted_scores_testdata['Logistic_Regression_Score'].map(score_mapping)
predicted_scores_testdata['XGBoost_Score_Category'] = predicted_scores_testdata['XGBoost_Score'].map(score_mapping)

# Print out the predicted loan scores
print("Predicted loan scores using Logistic Regression model:")
print(predicted_scores_testdata[['user_id', 'loan_id', 'Logistic_Regression_Score', 'Logistic_Regression_Score_Category']])

print("Predicted loan scores using XGBoost model:")
print(predicted_scores_testdata[['user_id', 'loan_id', 'XGBoost_Score', 'XGBoost_Score_Category']])


In [ ]:
predicted_scores_testdata.head()

In [ ]:
predicted_scores_testdata.to_csv('updated_new_predicted_scores2.csv', index=False)

In [ ]:
import os
print(os.getcwd())
